<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/Token_attention_with_head_importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [5]:
pip install datasets --quiet

     |████████████████████████████████| 362 kB 7.4 MB/s 
     |████████████████████████████████| 140 kB 64.2 MB/s 
     |████████████████████████████████| 101 kB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 15.5 MB/s 
     |████████████████████████████████| 212 kB 59.5 MB/s 
     |████████████████████████████████| 596 kB 42.6 MB/s 
     |████████████████████████████████| 127 kB 60.5 MB/s 
     |████████████████████████████████| 144 kB 48.4 MB/s 
     |████████████████████████████████| 271 kB 56.8 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.0 MB/s 
     |████████████████████████████████| 6.6 MB 35.1 MB/s 


Import Dataset and Model

In [7]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

Import the Reserach Papers dataset

In [8]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')

def longformer_finetuned_papers():
    model = AutoModelForSequenceClassification.from_pretrained('danielhou13/longformer-finetuned_papers', num_labels = 2)
    return model

def preprocess_function(tokenizer, example, max_length):
    example.update(tokenizer(example['text'], padding='max_length', max_length=max_length, truncation=True))
    return example

def get_papers_dataset(dataset_type):
    max_length = 2048
    dataset = load_dataset("danielhou13/cogs402dataset")[dataset_type]

    # tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    dataset = dataset.map(lambda x: preprocess_function(tokenizer, x, max_length), batched=True)
    setattr(dataset, 'input_columns', ['input_ids', 'attention_mask'])
    setattr(dataset, 'target_columns', ['labels'])
    setattr(dataset, 'max_length', max_length)
    setattr(dataset, 'tokenizer', tokenizer)
    return dataset

def papers_test_set():
    return get_papers_dataset('test')

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Import the news dataset

In [9]:
# def preprocess_function(tokenizer, example, max_length):
#     example.update(tokenizer(example['text'], padding='max_length', max_length=max_length, truncation=True))
#     return example

# def longformer_finetuned_news():
#     model = AutoModelForSequenceClassification.from_pretrained('danielhou13/longformer-finetuned-news-cogs402', num_labels = 2)
#     return model

# def get_news_dataset(dataset_type):
#     max_length = 2048
#     dataset = load_dataset("danielhou13/cogs402dataset2")[dataset_type]

#     tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')
#     dataset = dataset.map(lambda x: preprocess_function(tokenizer, x, max_length), batched=True)

#     labels = map(int, dataset['hyperpartisan'])
#     print(type(dataset['hyperpartisan']))
#     labels = list(labels)
#     dataset = dataset.add_column("labels", labels)

#     dataset = dataset.remove_columns(['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'])
#     print(dataset)
#     setattr(dataset, 'input_columns', ['input_ids', 'attention_mask'])
#     setattr(dataset, 'target_columns', ['labels'])
#     setattr(dataset, 'max_length', max_length)
#     setattr(dataset, 'tokenizer', tokenizer)
#     return dataset

# def news_train_set():
#     return get_news_dataset('train')

# def news_test_set():
#     return get_news_dataset('validation')

Load papers model and dataset and preprocess it

In [10]:
cogs402_test = papers_test_set()
model = longformer_finetuned_papers()
columns = cogs402_test.input_columns + cogs402_test.target_columns
print(columns)
cogs402_test.set_format(type='torch', columns=columns)
cogs402_test=cogs402_test.remove_columns(['text'])

Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

Using custom data configuration danielhou13--cogs402dataset-cc784554b797f843


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

...

Parameter 'function'=<function get_papers_dataset.<locals>.<lambda> at 0x7f25d3b160e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/567M [00:00<?, ?B/s]

['input_ids', 'attention_mask', 'labels']


Load news model and dataset and preprocess it

In [11]:
# cogs402_test = news_test_set()
# model = longformer_finetuned_news()
# columns = cogs402_test.input_columns + cogs402_test.target_columns
# print(columns)
# cogs402_test.set_format(type='torch', columns=columns)

In [12]:
if torch.cuda.is_available():
    model = model.cuda()

print(model.device)

cuda:0


Predict using the model on the selected dataset

In [13]:
from transformers import Trainer, TrainingArguments

batch_size = 1
gradient_acc = 4
model_name = f"longformer-finetuned_papers"
training_args = TrainingArguments(output_dir=f"models/{model_name}",
                                  num_train_epochs = 2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  push_to_hub=False,
                                  log_level="error",
                                  fp16=True,
                                  gradient_accumulation_steps=gradient_acc,
                                  gradient_checkpointing=True,
                                  save_strategy = "epoch")

F1 and accuracy are good general metrics for model performance. Recall and precision can be used if we require low false negatives or false positives.

In [14]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [15]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator = data_collator
)

In [16]:
preds_output = trainer.predict(cogs402_test)

We want to find the false negatives, and false positives

In [17]:
y_preds = np.argmax(preds_output.predictions, axis=1)
y_true = np.array(cogs402_test["labels"])

In [18]:
diff = y_true-y_preds
correct = np.where(diff == 0)[0]
incorrect = np.where(diff != 0)[0]
false_pos = np.where(diff == -1)[0]
false_neg = np.where(diff == 1)[0]
print('Correctly classified: ', np.where(diff == 0)[0])
print('Incorrectly classified: ', np.where(diff != 0)[0])
print('False positives: ', np.where(diff == -1)[0])
print('False negatives: ', np.where(diff == 1)[0])

Correctly classified:  [   0    1    2 ... 1176 1177 1178]
Incorrectly classified:  [   8   31   59   88  110  112  124  150  154  189  199  213  218  231
  238  245  252  299  302  316  333  335  337  347  353  357  362  372
  376  383  390  396  403  411  418  424  435  437  441  456  468  469
  496  497  520  538  553  601  610  622  633  638  649  659  685  708
  732  756  778  782  785  790  799  806  810  814  820  824  832  846
  877  909  912  937  941  944  965  979  985  990 1003 1005 1039 1050
 1060 1108 1149 1150 1151]
False positives:  [  88  252  299  302  337  362  435  468  469  601  806 1108]
False negatives:  [   8   31   59  110  112  124  150  154  189  199  213  218  231  238
  245  316  333  335  347  353  357  372  376  383  390  396  403  411
  418  424  437  441  456  496  497  520  538  553  610  622  633  638
  649  659  685  708  732  756  778  782  785  790  799  810  814  820
  824  832  846  877  909  912  937  941  944  965  979  985  990 1003
 1005 1039

Take example for evaluation

In [19]:
rand_fp = np.random.choice(false_pos, size=1)
rand_fn = np.random.choice(false_neg, size=1)
rand_correct = np.random.choice(correct, size=2)
testexam = cogs402_test[rand_fp]

In [20]:
print(testexam)

{'labels': tensor([0]), 'input_ids': tensor([[    0, 49714,  3923,  ...,  4405,    17,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


In [21]:
output = model(testexam["input_ids"].cuda(), attention_mask=testexam['attention_mask'].cuda(), labels=testexam['labels'].cuda(), output_attentions = True)
batch_attn = output[-2]
output_attentions = torch.stack(batch_attn).cpu()
global_attention = output[-1]
output_global_attentions = torch.stack(global_attention).cpu()
print("output_attention.shape", output_attentions.shape)
print("gl_output_attention.shape", output_global_attentions.shape)

output_attention.shape torch.Size([12, 1, 12, 2048, 514])
gl_output_attention.shape torch.Size([12, 1, 12, 2048, 1])


In [79]:
print(testexam['labels'][0])
print(output[1].argmax())

tensor(0)
tensor(1, device='cuda:0')


In [22]:
# print(os.getcwd())
# yes = torch.load("resources/longformer_test2/epoch_3/aggregate_attn.pt")

Convert sliding attention matrix to correct seq_len x seq_len matrix

In [23]:
def create_head_matrix(output_attentions, global_attentions):
    new_attention_matrix = torch.zeros((output_attentions.shape[0], 
                                      output_attentions.shape[0]))
    for i in range(output_attentions.shape[0]):
        test_non_zeroes = torch.nonzero(output_attentions[i]).squeeze()
        test2 = output_attentions[i][test_non_zeroes[1:]]
        new_attention_matrix_indices = test_non_zeroes[1:]-257 + i
        new_attention_matrix[i][new_attention_matrix_indices] = test2
        new_attention_matrix[i][0] = output_attentions[i][0]
        new_attention_matrix[0] = global_attentions.squeeze()[:output_attentions.shape[0]]
    return new_attention_matrix


def attentions_all_heads(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = create_head_matrix(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_batches(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = attentions_all_heads(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_layers(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = all_batches(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

In [24]:
converted_mat = all_layers(output_attentions, output_global_attentions).detach().cpu().numpy()
print(converted_mat.shape)

(12, 1, 12, 2048, 2048)


In [25]:
print(testexam['input_ids'])

tensor([[    0, 49714,  3923,  ...,  4405,    17,     2]])


In [26]:
all_tokens = tokenizer.convert_ids_to_tokens(testexam["input_ids"][0])

Load head importance model and scale the attentions by head importance

In [27]:
head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/pretrained/head_importance.pt")

In [28]:
def scale_by_importance(attention_matrix, head_importance):
  new_matrix = np.zeros_like(attention_matrix)
  for i in range(attention_matrix.shape[0]):
    head_importance_layer = head_importance[i]
    for j in range(attention_matrix.shape[1]):
      new_matrix[i,j] = attention_matrix[i,j] * np.expand_dims(head_importance_layer, axis=(1,2))
  return new_matrix

In [29]:
converted_mat_importance = scale_by_importance(converted_mat, head_importance)

Lets suppose we want the topk attended tokens for each token in each head, batch and layer.

In [30]:
# get the top k and indexes and values for each row
def find_top_attention_unsummed(scores_mat, k):
  indices = scores_mat.argsort(axis=4)[:, :, :, :, :-(k+1):-1]
  vals = np.take_along_axis(scores_mat, indices, axis=4)  
  return indices, vals

#find the tokens using the index matrix and the all_tokens list to create a 
#matrix of tokens 
def get_tokens(index_matrix, example=None):
  highest_tokens = []
  #layer
  for i in range(index_matrix.shape[0]):
    row_tokens = []
    #batch
    for j in range(index_matrix.shape[1]):
      batch_tokens = []

      if (example is not None) and (j != example):
        continue

      #head
      for k in range(index_matrix.shape[2]):
        head_tokens = []
        #token
        for x in range(index_matrix.shape[3]):
          tokens = [all_tokens[idx] for idx in index_matrix[i][j][k][x]]
          head_tokens.append(tokens)
        batch_tokens.append(head_tokens)
      row_tokens.append(batch_tokens)
    highest_tokens.append(row_tokens)
  return np.array(highest_tokens)

#format into a dataframe
def highest_attended_tokens(index, values, tokens, all_tokens, example=None):
    dataframe=[]
    for i in range(index.shape[0]):
      for j in range(index.shape[1]):

        if (example is not None) and (j != example):
          continue

        for k in range(index.shape[2]):
          for x in range(index.shape[3]):
            for y in range(index.shape[4]):
              d = {"token":all_tokens[x], 'self_position':x, 
                  "attended_token": tokens[i,j,k,x,y],
                  'token_position':index[i,j,k,x,y], 
                  'attention_scores':values[i,j,k,x,y],
                  'layer':(i+1), 'head':(k+1),
                  'rank':(y+1),
                  'batch':j}
              dataframe.append(d)
    df = pd.DataFrame(dataframe)
    return df

In [31]:
#combine the previous functions
def highest_tokens(matrix, k, all_tokens, example=None):
  index, values = find_top_attention_unsummed(matrix, k)
  highest_tokens = get_tokens(index, example)
  df = highest_attended_tokens(index, values, highest_tokens, all_tokens, example)
  return df

Thus, for every token, we can get the top k tokens that this token attends to. We can filter by batch, layer, head, rank, position, etc.,. The downsides are that its not very visually appealing despite being organized.

In [32]:
df2 = highest_tokens(converted_mat_importance, 10, all_tokens, 0)
df2

,token,self_position,attended_token,token_position,attention_scores,layer,head,rank,batch
0,<s>,0,<s>,0,0.002403,1,1,1,0
1,<s>,0,Ġthen,512,0.001279,1,1,2,0
2,<s>,0,Ġif,1024,0.001076,1,1,3,0
3,<s>,0,Ġin,1536,0.001067,1,1,4,0
4,<s>,0,Ġprob,291,0.000076,1,1,5,0
...,...,...,...,...,...,...,...,...,...
2949115,</s>,2047,Ġregression,2028,0.000060,12,12,6,0
2949116,</s>,2047,</s>,2047,0.000058,12,12,7,0
2949117,</s>,2047,Ġruns,2043,0.000056,12,12,8,0
2949118,</s>,2047,.,2041,0.000028,12,12,9,0


Get the sum of the attentions for all the tokens (column-wise). In other words, find out how much every word is attended to

In [33]:
attention_matrix_importance = converted_mat_importance.sum(axis=3)
print(attention_matrix_importance.shape)

(12, 1, 12, 2048)


Get top k attended words for each head, for each example in batch, for each layer

In [34]:
def find_top_attention(scores_mat, k):
  indices = scores_mat.argsort(axis=3)[:, :, :, :-(k+1):-1]
  vals = np.take_along_axis(scores_mat, indices, axis=3)
  return indices, vals

We want the position (index) of the token, the attention value, and the actual token itself.

In [35]:
def get_tokens2(index_matrix, example=None):
  highest_tokens = []
  #layer
  for i in range(index_matrix.shape[0]):
    row_tokens = []
    #batch
    for j in range(index_matrix.shape[1]):
      batch_tokens = []

      if example is not None and j != example:
        continue

      #head
      for k in range(index_matrix.shape[2]):
        tokens = [all_tokens[idx] for idx in index_matrix[i][j][k]]
        batch_tokens.append(tokens)
      row_tokens.append(batch_tokens)
    highest_tokens.append(row_tokens)
  return np.array(highest_tokens)

In [36]:
def highest_attended_dataframe(index, values, tokens, example=None):
    dataframe=[]
    #layer
    for i in range(index.shape[0]):
      #batch
      for j in range(index.shape[1]):
        if example is not None and j != example:
          continue
        #head
        for k in range(index.shape[2]):
          #token
          for x in range(index.shape[3]):
            d = {"token":tokens[i,j,k,x], 'position':index[i,j,k,x], 
                'attention_scores':values[i,j,k,x],
                 'layer':(i+1), 'head':(k+1),
                 'rank':(x+1),
                 'batch':j}
            dataframe.append(d)
    df = pd.DataFrame(dataframe)
    return df

In [37]:
def highest_attentions_summed(matrix, k, all_tokens, example=None):
  index, values = find_top_attention(matrix, k)
  highest_tokens = get_tokens2(index, example)
  df_highest = highest_attended_dataframe(index, values, highest_tokens, example)
  return df_highest

In [38]:
df_highest = highest_attentions_summed(attention_matrix_importance, 10, all_tokens, 0)
df_highest

,token,position,attention_scores,layer,head,rank,batch
0,<s>,0,1.033246,1,1,1,0
1,Ġif,1024,0.279133,1,1,2,0
2,Ġthen,512,0.226342,1,1,3,0
3,Ġin,1536,0.213620,1,1,4,0
4,ĠThere,685,0.120233,1,1,5,0
...,...,...,...,...,...,...,...
1435,.,1891,0.049180,12,12,6,0
1436,.,472,0.047793,12,12,7,0
1437,.,492,0.043417,12,12,8,0
1438,.,1987,0.042563,12,12,9,0


Get the attention of a token at a position for each layer and head. Take one example at a time as each example has different tokens. Pros: can isolate for layers and/or heads. Cons: not much context for the attention scores

In [39]:
def position_attention(agg_matrix, position, example=None):
  dataframe=[]
  if example is not None:
    new_mat = agg_matrix[:, example, :]
    new_mat = new_mat.squeeze()
    #layer
    for i in range(new_mat.shape[0]):
      #head
      for j in range(new_mat.shape[1]):
        temp = new_mat[i,j].argsort()[::-1]
        temp = np.where(temp==position)[0].squeeze() + 1
        d = {"token":all_tokens[position], 'position':position, 
            'attention_scores':new_mat[i,j,position], 'layer':(i+1), 'head':(j+1),
            'rank':temp, 'batch':example}
        dataframe.append(d)
  else:
    new_mat = agg_matrix
    #layer
    for i in range(new_mat.shape[0]):
      #batch
      for j in range(new_mat.shape[1]):
        #head
        for k in range(new_mat.shape[2]):
          temp = new_mat[i,j,k].argsort()[::-1]
          temp = np.where(temp==position)[0].squeeze() + 1
          d = {"token":all_tokens[position], 'position':position, 
              'attention_scores':new_mat[i,j,k,position], 'layer':(i+1), 'head':(k+1),
              'rank':temp, 'batch':j}
          dataframe.append(d)
  df = pd.DataFrame(dataframe)
  return df

In [40]:
position_df = position_attention(attention_matrix_importance, 0)
position_df[position_df["head"]==1]

,token,position,attention_scores,layer,head,rank,batch
0,<s>,0,1.033246,1,1,1,0
12,<s>,0,6.664244,2,1,1,0
24,<s>,0,2.211680,3,1,1,0
36,<s>,0,9.574983,4,1,1,0
48,<s>,0,0.869655,5,1,1,0
60,<s>,0,0.830983,6,1,12,0
72,<s>,0,0.112237,7,1,127,0
84,<s>,0,0.271344,8,1,1,0
96,<s>,0,0.000834,9,1,39,0
108,<s>,0,0.000365,10,1,123,0


If really needed, can just have the full matrix of the position, ranks, attention scores, layers, and heads of each token per example. 

Tokens are all the tokens in the example. Position is the location of the token with zero-based indexing. Attention scores are the aggregate, scaled attention scores.

Layer goes from 1 to 12.
Head goes from 1 to 12.

Rank is the attention score rank with respect to layer and head. Goes from 1 to number of tokens in the example

Pros: can search up whatever is needed. Has access to all the information and can be extracted for comparisons Cons: have to know what you want and manually look it up

In [41]:
def full_matrix(agg_matrix, example=None):
  dataframe=[]

  if example is not None:
    new_mat = agg_matrix[:, example]
    new_mat = new_mat.squeeze()
    print(new_mat.shape)

    #layer
    for i in range(new_mat.shape[0]):
      #head
      for j in range(new_mat.shape[1]):
        temp = new_mat[i,j].argsort()[::-1]      
        #token
        for k in range(new_mat.shape[2]):
          temp2 = np.where(temp==k)[0].squeeze() + 1
          d = {"token":all_tokens[k], 'position':k, 
              'attention_scores':new_mat[i,j,k], 'layer':(i+1), 'head':(j+1),
              'rank':temp2}
          dataframe.append(d)
  else:
    new_mat = agg_matrix
    print(new_mat.shape)
    
    #layer
    for i in range(new_mat.shape[0]):
      #batch
      for j in range(new_mat.shape[1]):
        #head
        for k in range(new_mat.shape[2]):
          temp = new_mat[i,j,k].argsort()[::-1]      
          #token
          for x in range(new_mat.shape[3]):
            temp2 = np.where(temp==x)[0].squeeze() + 1
            d = {"token":all_tokens[x], 'position':x, 
                'attention_scores':new_mat[i,j,k,x], 'layer':(i+1), 'head':(k+1),
                'rank':temp2, 'batch':j}
            dataframe.append(d)
  df = pd.DataFrame(dataframe)
  return df

In [42]:
full_mat = full_matrix(attention_matrix_importance)

(12, 1, 12, 2048)


In [43]:
full_mat[(full_mat['head']==11) & (full_mat['layer']==11)]

,token,position,attention_scores,layer,head,rank,batch
266240,<s>,0,0.000806,11,11,1012,0
266241,Debug,1,0.011483,11,11,272,0
266242,ging,2,0.001445,11,11,824,0
266243,ĠMachine,3,0.004140,11,11,527,0
266244,ĠLearning,4,0.009419,11,11,317,0
...,...,...,...,...,...,...,...
268283,Ġruns,2043,0.000573,11,11,1117,0
268284,ĠMall,2044,0.000379,11,11,1266,0
268285,ory,2045,0.000185,11,11,1551,0
268286,âĢ,2046,0.000034,11,11,2018,0


In [73]:
## convert the text/attention list to latex code, which will further generates the text heatmap based on attention weights.
import numpy as np

latex_special_token = ["!@#$%^&*()"]

def generate(text_list, attention_list, latex_file, color='red', rescale_value = False):
	assert(len(text_list) == len(attention_list))
	if rescale_value:
		attention_list = rescale(attention_list)
	word_num = len(text_list)
	text_list = clean_word(text_list)
	with open(latex_file,'w') as f:
		f.write(r'''\documentclass[varwidth]{standalone}
\special{papersize=210mm,297mm}
\usepackage{color}
\usepackage{tcolorbox}
\usepackage{CJK}
\usepackage{adjustbox}
\tcbset{width=0.9\textwidth,boxrule=0pt,colback=red,arc=0pt,auto outer arc,left=0pt,right=0pt,boxsep=5pt}
\begin{document}
\begin{CJK*}{UTF8}{gbsn}'''+'\n')
		string = r'''{\setlength{\fboxsep}{0pt}\colorbox{white!0}{\parbox{0.9\textwidth}{'''+"\n"
		for idx in range(word_num):
			string += "\\colorbox{%s!%s}{"%(color, attention_list[idx])+"\\strut " + text_list[idx]+"} "
		string += "\n}}}"
		f.write(string+'\n')
		f.write(r'''\end{CJK*}
\end{document}''')

def rescale(input_list):
	the_array = np.asarray(input_list)
	the_max = np.max(the_array)
	the_min = np.min(the_array)
	rescale = (the_array - the_min)/(the_max-the_min)*100
	return rescale.tolist()


def clean_word(word_list):
	new_word_list = []
	for word in word_list:
		for latex_sensitive in ["\\", "%", "&", "^", "#", "_",  "{", "}"]:
			if latex_sensitive in word:
				word = word.replace(latex_sensitive, '\\'+latex_sensitive)
		new_word_list.append(word)
	return new_word_list

In [68]:
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [76]:
print(all_tokens)
average_attention = attention_matrix_importance.squeeze().sum(axis=1)
average_attention = average_attention.sum(axis=0)
average_attention = normalize(average_attention)
print(rand_fp)

['<s>', 'Debug', 'ging', 'ĠMachine', 'ĠLearning', 'ĠT', 'asks', 'Ġ', 'Ġar', 'X', 'iv', ':', '16', '03', '.', '07', '292', 'v', '1', 'Ġ[', 'cs', '.', 'LG', ']', 'Ġ23', 'ĠMar', 'Ġ2016', 'Ġ', 'ĠAle', 'ks', 'and', 'ar', 'ĠChak', 'arov', 'A', 'Ġ,', 'ĠAd', 'ity', 'a', 'ĠNor', 'i', 'B', 'Ġ,', 'ĠSr', 'ir', 'am', 'ĠRaj', 'am', 'ani', 'B', 'Ġ,', 'ĠShay', 'ak', 'ĠSen', 'C', 'Ġ,', 'Ġand', 'ĠDeep', 'ak', 'ĠVij', 'ay', 'ke', 'er', 'thy', 'D', 'ĠA', 'Ġ', 'ĠUniversity', 'Ġof', 'ĠColorado', ',', 'ĠBoulder', 'ĠB', 'ĠMicrosoft', 'ĠResearch', 'ĠC', 'ĠCarnegie', 'ĠMellon', 'ĠUniversity', 'ĠD', 'ĠIBM', 'ĠResearch', 'ĠMarch', 'Ġ24', ',', 'Ġ2016', 'ĠAbstract', 'Ġ', 'ĠUnlike', 'Ġtraditional', 'Ġprograms', 'Ġ(', 'such', 'Ġas', 'Ġoperating', 'Ġsystems', 'Ġor', 'Ġword', 'Ġprocessors', ')', 'Ġwhich', 'Ġhave', 'Ġlarge', 'Ġamounts', 'Ġof', 'Ġcode', ',', 'Ġmachine', 'Ġlearning', 'Ġtasks', 'Ġuse', 'Ġprograms', 'Ġwith', 'Ġrelatively', 'Ġsmall', 'Ġamounts', 'Ġof', 'Ġcode', 'Ġ(', 'written', 'Ġin', 'Ġmachine', 'Ġlearning'

In [84]:
generate(all_tokens, (average_attention*100), "papers_435.tex", 'red')

In [82]:
print(attention_matrix_importance[11].squeeze().shape)
average_attention_final_layer = attention_matrix_importance[11].squeeze().sum(axis=0)
average_attention_final_layer = normalize(average_attention_final_layer)

(12, 2048)


In [86]:
generate(all_tokens, (average_attention_final_layer*100), "papers_435_layer_11_only.tex", 'red')